In [ ]:
# ==========================================================
# MobileNetV2 Transfer-Learning   (2- or 3-class dataset)
# ==========================================================
import tensorflow as tf, pathlib, zipfile
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

ZIP_PATH  = "gesture_data.zip"          # <-- upload your zip
DATA = pathlib.Path("gesture_data")
IMG_SIZE  = (160, 160)                       # MobileNetV2 default
BATCH     = 32
EPOCHS_H  = 10                               # head-only
EPOCHS_FT = 5                                # fine-tune
PATIENCE  = 3

# ---------------- 1  unzip & count classes ----------------
zipfile.ZipFile(ZIP_PATH).extractall(DATA)
DATA_ROOT = pathlib.Path("gesture_data/gesture_data")
class_names  = sorted([p.name for p in DATA_ROOT.iterdir() if p.is_dir()])
n_classes    = len(class_names)
print("Classes:", class_names, "\nTotal folders:", n_classes)

Classes: ['background', 'thumbs_up', 'v_sign'] 
Total folders: 3


In [ ]:
# ---------------- 2  data loader w/ gentle aug ------------
datagen = ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    brightness_range=[0.7, 1.3],
    zoom_range=0.10
)

train_gen = datagen.flow_from_directory(
    DATA_ROOT, target_size=IMG_SIZE, batch_size=BATCH,
    class_mode="sparse", subset="training")

val_gen = datagen.flow_from_directory(
    DATA_ROOT, target_size=IMG_SIZE, batch_size=BATCH,
    class_mode="sparse", subset="validation")

Found 360 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [ ]:
# ---------------- 3  MobileNetV2 backbone -----------------
base = tf.keras.applications.MobileNetV2(
           include_top=False, weights="imagenet",
           input_shape=IMG_SIZE+(3,),
           alpha=1.0)                       # full-width network
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.25),
    layers.Dense(n_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

es = EarlyStopping(patience=PATIENCE, restore_best_weights=True,
                   monitor="val_accuracy", verbose=1)
ck = ModelCheckpoint("best_mnet_head.keras", save_best_only=True,
                     monitor="val_accuracy", verbose=1)

hist_head = model.fit(train_gen, validation_data=val_gen,
                      epochs=EPOCHS_H, callbacks=[es, ck])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 835ms/step - accuracy: 0.3519 - loss: 1.3559
Epoch 1: val_accuracy improved from -inf to 0.73333, saving model to best_mnet_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3631 - loss: 1.3347 - val_accuracy: 0.7333 - val_loss: 0.5941
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 904ms/step - accuracy: 0.7536 - loss: 0.5889
Epoch 2: val_accuracy improved from 0.73333 to 0.83333, saving model to best_mnet_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7553 - loss: 0.5841 - val_accuracy: 0.8333 - val_loss: 0.3828
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 843ms/step - accuracy: 0.8804 - loss: 0.3401
Epoch 3: val_accuracy improved from 0.83333 to 0.84444, saving model to best_mnet_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8815 - loss: 0.3382 - val_accuracy: 0.8444 - val_loss: 0.2984
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - accuracy: 0.8925 - loss: 0.2602
Epoch 4: val_accuracy improved from

In [ ]:
# ---------------- 4  optional fine-tune last 20 layers ----
base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ck_ft = ModelCheckpoint("best_mnet_ft.keras", save_best_only=True,
                        monitor="val_accuracy", verbose=1)
es_ft = EarlyStopping(patience=PATIENCE, restore_best_weights=True,
                      monitor="val_accuracy", verbose=1)

hist_ft = model.fit(train_gen, validation_data=val_gen,
                    epochs=EPOCHS_FT, callbacks=[es_ft, ck_ft])

Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 953ms/step - accuracy: 0.7762 - loss: 0.4687
Epoch 1: val_accuracy improved from -inf to 0.94444, saving model to best_mnet_ft.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7778 - loss: 0.4662 - val_accuracy: 0.9444 - val_loss: 0.1697
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 977ms/step - accuracy: 0.8586 - loss: 0.3326
Epoch 2: val_accuracy did not improve from 0.94444
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.8597 - loss: 0.3312 - val_accuracy: 0.9222 - val_loss: 0.1927
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 942ms/step - accuracy: 0.8614 - loss: 0.3186
Epoch 3: val_accuracy improved from 0.94444 to 0.96667, saving model to best_mnet_ft.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8646 - loss: 0.3135 - val_accuracy: 0.9667 - val_loss: 0.1254
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 935ms/step - accuracy: 0.9333 - loss: 0.1984
Epoch 4: val_accuracy improved from 0.96667 to 0.97778, saving model to best_mnet_ft.

In [ ]:
# ---------------- 5  export to TFLite ---------------------
converter   = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_mnet = converter.convert()
open("mobilenet_gestures.tflite", "wb").write(tflite_mnet)

print("\n✅ MobileNetV2 training done.  Final val-accuracy:",
      hist_ft.history.get("val_accuracy", hist_head.history["val_accuracy"])[-1])
print("📦 File saved: mobilenet_gestures.tflite")

Saved artifact at '/tmp/tmp0pq0a2f3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140058820419920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896524944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896528400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896528016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896526864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140058820421456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896527824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896523024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896528784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140055896528592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1400558965